## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import dataset_comp
import curve_fit

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(curve_fit)

# Relation Between $M_{200}$ and $v_\mathrm{max}$

NFW halos have a tight log-linear relation between these two measures of the halo mass, as the varying concentrations are noticable mostly closer to the centre than at $r_\mathrm{max}$. 

Here, I plot the $M_{200}$ against the $v_\mathrm{max}$ of each isolated halo in the LCDM simulation and construct a linear fit. 

In [ ]:
snap_id = 127
sim_id = 'V1_MR_fix'

m31_id = (1,0)
mw_id = (2,0)

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
snap = snapshot.Snapshot(sim_id, snap_id)

m200 = snap.get_subhalos('Group_M_TopHat200', h5_group='FOF')[2:] \
                * units.g.to(units.Msun)
sub_idx = snap.get_subhalos('FirstSubhaloID', h5_group='FOF').astype(int)[2:]

max_point = snap.get_subhalos("Max_Vcirc", "Extended")
vmax_sub = max_point[:,0] * units.cm.to(units.km)

# Select within 2 Mpc:
centre = dataset_comp.compute_LG_centre(snap, m31_id, mw_id)
gcop = snap.get_subhalos('GroupCentreOfPotential', h5_group='FOF')[2:]
gcop = dataset_comp.periodic_wrap(snap, centre, gcop)
gcop = np.linalg.norm(gcop - centre, axis=1) * units.cm.to(units.kpc)
mask = dataset_comp.within_distance_range(gcop, 0, 2000)
mask = np.logical_and(mask, m200 > 0)

m200 = m200[mask]
sub_idx = sub_idx[mask]
vmax = vmax_sub[sub_idx]

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("$\log_{10}(v_\mathrm{max} / \mathrm{km/s})$")
ax.set_ylabel("$\log_{10}(M_{200} / \mathrm{M}_\odot)$")

ax.set_title("Masses of Isolated Subhalos")

x = np.log10(vmax)
y = np.log10(m200)
ax.scatter(x, y)

linfit_params = np.polyfit(x, y, 1)
alpha = linfit_params[0]
c = linfit_params[1]
print(alpha, c)

x_fit = np.linspace(np.min(x), np.max(x), 10)
y_fit = alpha * x_fit + c
ax.plot(x_fit, y_fit, c='red')

In [ ]:
print(((np.log10(5*10**7) - c) / alpha))
print(10**((np.log10(5*10**7) - c) / alpha))

In [ ]:
filename = "M200_vs_vmax.png"
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')